In [ ]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import pandas as pd
from tqdm import tqdm

#Credientials 
PLAYLIST_ID = "PLx00VbyG3uL8dICyMXX1vLhRDsNR3o8gA"
info = {
    "client_id": client_id,
    "client_secret": client_secret,
    "refresh_token": refresh_token
}
creds = Credentials.from_authorized_user_info(info=info)
service = build("youtube", "v3", credentials=creds)

#Creates request and outputs videos. 
request = service.playlistItems().list(
    playlistId=PLAYLIST_ID,
    part="id,snippet",
    maxResults=1000
)
response = request.execute()

# Create an empty dataframe
df = pd.DataFrame(columns=["title", "view_count", "like_count", "comment_count","published", "duration"])

# Iterates through the list of videos
while True:
    for item in tqdm(response["items"]):
        video_id = item["snippet"]["resourceId"]["videoId"]
        video_title = item["snippet"]["title"]
        published = item["snippet"]["publishedAt"]

        #Requests stats for each video within list. 
        video_request = service.videos().list(
            id=video_id,
            part="id, snippet, statistics, contentDetails"
        )
        video_response = video_request.execute()
        

        if video_response["items"]:
            # Get the view count, like count, and comment count for the video
            view_count = video_response["items"][0]["statistics"]["viewCount"]
            like_count = video_response["items"][0]["statistics"]["likeCount"]
            comment_count = video_response["items"][0]["statistics"]["commentCount"]
            duration = video_response["items"][0]["contentDetails"]["duration"]
      

        #Adds video data to dataframe
        df = df.append(
            {"title": video_title, "view_count": view_count, "like_count": like_count, "comment_count": comment_count,"published":published, "duration": duration,"video_id":video_id},
            ignore_index=True
        )

    #Check if there are more pages of results
    if "nextPageToken" in response:
        # If there are more pages, retrieve the next page
        request = service.playlistItems().list(
            playlistId=PLAYLIST_ID,
            part="id,snippet",
            maxResults=1000,
            pageToken=response["nextPageToken"]
        )
        response = request.execute()
    else:
        # If there are no more pages, break out of the loop
        break

100%|██████████| 1/1 [00:00<00:00, 40.51it/s]

                                                  title view_count like_count  \
0     Barstool Pizza Review - No Name Pub (Big Pine ...     182246       4929   
1     Barstool Pizza Review - Coast To Coast Pizza C...     194036       5743   
2     Barstool Pizza Review - Enrico’s Pizza (Tavern...     210309       4961   
3     Barstool Pizza Review - The Pizza Slice (Key L...     187891       5707   
4     Barstool Pizza Review - Pedroso's Pizza The Tr...     228351       5639   
...                                                 ...        ...        ...   
1346                                      Private video     112506        463   
1347                                      Private video     112506        463   
1348                                      Private video     112506        463   
1349                                      Private video     112506        463   
1350                                      Private video     112506        463   

     comment_count         

In [ ]:
df

,title,view_count,like_count,comment_count,published,duration,video_id
0,Barstool Pizza Review - No Name Pub (Big Pine ...,182246,4929,574,2023-01-20T18:17:53Z,PT2M55S,8TidCfOIruk
1,Barstool Pizza Review - Coast To Coast Pizza C...,194036,5743,549,2023-01-19T18:07:30Z,PT3M5S,5PRP9O2dPRE
2,Barstool Pizza Review - Enrico’s Pizza (Tavern...,210309,4961,665,2023-01-18T05:27:07Z,PT3M34S,uN8O_p9KYiQ
3,Barstool Pizza Review - The Pizza Slice (Key L...,187891,5707,948,2023-01-17T06:12:39Z,PT2M11S,f-Te38Zi6tY
4,Barstool Pizza Review - Pedroso's Pizza The Tr...,228351,5639,610,2023-01-16T17:04:49Z,PT4M12S,5vvOeVX3Euc
...,...,...,...,...,...,...,...
1346,Private video,112506,463,58,2020-03-03T18:09:50Z,PT1M10S,KArc1Hc7PKA
1347,Private video,112506,463,58,2020-03-03T18:09:50Z,PT1M10S,NHHbQRVQJMo
1348,Private video,112506,463,58,2020-03-03T18:10:00Z,PT1M10S,NeNOVE-aYEw
1349,Private video,112506,463,58,2023-01-13T21:35:46Z,PT1M10S,WKK1WS3e3kQ


In [ ]:
from datetime import datetime, timedelta
import re

def convert(time_string):
  try:
    match = re.search(r'PT(\d+)M(\d+)S', time_string)
    minutes = int(match.group(1))
    seconds = int(match.group(2))
    time = timedelta(minutes=minutes, seconds=seconds)
    total_seconds = time.total_seconds()
    return(total_seconds)
  except:
    return(time_string)

datetime = []
for iso in tqdm(df['duration'][:50]):
  datetime.append(convert(iso))
datetime



100%|██████████| 50/50 [00:00<00:00, 25735.08it/s]


[131.0,
 252.0,
 383.0,
 178.0,
 631.0,
 201.0,
 227.0,
 'PT58S',
 257.0,
 147.0,
 227.0,
 119.0,
 155.0,
 173.0,
 274.0,
 260.0,
 233.0,
 201.0,
 381.0,
 187.0,
 395.0,
 158.0,
 268.0,
 253.0,
 293.0,
 150.0,
 288.0,
 304.0,
 166.0,
 297.0,
 311.0,
 193.0,
 182.0,
 359.0,
 234.0,
 242.0,
 546.0,
 189.0,
 139.0,
 190.0,
 605.0,
 163.0,
 238.0,
 220.0,
 159.0,
 160.0,
 167.0,
 222.0,
 431.0,
 364.0]

In [ ]:
from datetime import datetime, timedelta
import re
def conv(time_string):
  try:
    match = re.search(r'PT((\d+)M)?((\d+)S)?', time_string)
    minutes = int(match.group(2) or 0)
    seconds = int(match.group(4) or 0)
    time = timedelta(minutes=minutes, seconds=seconds)
    total_seconds = time.total_seconds()
    return(total_seconds)
  except:
    return(time_string)
datetime = []
for iso in tqdm(df['duration']):
  datetime.append(conv(iso))
df["duration"] = datetime
  


In [ ]:
iso_dates = df['published']
dates = [datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ').date() for date in iso_dates]
formatted_dates = [date.strftime('%Y-%-m-%-d') for date in dates]
df['published'] = formatted_dates

In [ ]:
df

,title,view_count,like_count,comment_count,published,duration,video_id
0,Barstool Pizza Review - No Name Pub (Big Pine ...,182246,4929,574,2023-1-20,175.0,8TidCfOIruk
1,Barstool Pizza Review - Coast To Coast Pizza C...,194036,5743,549,2023-1-19,185.0,5PRP9O2dPRE
2,Barstool Pizza Review - Enrico’s Pizza (Tavern...,210309,4961,665,2023-1-18,214.0,uN8O_p9KYiQ
3,Barstool Pizza Review - The Pizza Slice (Key L...,187891,5707,948,2023-1-17,131.0,f-Te38Zi6tY
4,Barstool Pizza Review - Pedroso's Pizza The Tr...,228351,5639,610,2023-1-16,252.0,5vvOeVX3Euc
...,...,...,...,...,...,...,...
1346,Private video,112506,463,58,2020-3-3,70.0,KArc1Hc7PKA
1347,Private video,112506,463,58,2020-3-3,70.0,NHHbQRVQJMo
1348,Private video,112506,463,58,2020-3-3,70.0,NeNOVE-aYEw
1349,Private video,112506,463,58,2023-1-13,70.0,WKK1WS3e3kQ


In [ ]:
from google.colab import files
df.to_csv('output.csv', encoding = 'utf-8-sig') 
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

video_links = []
for id in video_ids:
  video_links.append("youtube.com/watch?v=" + id)
df["links"] = video_links
